In [1]:
import pandas as pd
import requests
import time
df_agri=pd.read_csv("../Cleaned CSVs/Mango-Raw-Ripe.csv")
df_agri['Price Date'] = pd.to_datetime(df_agri['Price Date'])
df_agri['District Name'].unique()

array(['Dharmapuri', 'Pudukkottai', 'Thiruvellore', 'Salem', 'Madurai',
       'Theni', 'Thiruvannamalai', 'Vellore', 'Ranipet', 'Ariyalur',
       'Virudhunagar', 'Kancheepuram', 'Chengalpattu', 'Cuddalore',
       'Dindigul', 'Krishnagiri', 'Sivaganga', 'Erode', 'Thirupur',
       'Villupuram', 'The Nilgiris', 'Kallakuruchi', 'Thirunelveli',
       'Karur', 'Thiruchirappalli', 'Tuticorin', 'Namakkal', 'Thanjavur',
       'Coimbatore', 'Nagapattinam', 'Thiruvarur',
       'Nagercoil (Kannyiakumari)', 'Thirupathur', 'Perambalur',
       'Ramanathapuram', 'Tenkasi'], dtype=object)

In [223]:
CROP_LOOKBACK_DAYS = {
    'Bajra(Pearl Millet/Cumbu)': 85+30,
    'Banana' : 120,
    'Banana - Green': 100,
    'Black Gram Dal (Urd Dal)': 85+30,
    'Cashewnuts': 90+30,
    'Coconut': 365,
    'Coriander(Leaves)': 120+30,
    'Cotton': 180+30,
    'Garlic': 270,
    'Green Gram Dal (Moong Dal)': 90+30,
    'Groundnut': 120+30,
    'Guava': 210,
    'Jowar(Sorghum)': 130+30,
    'Lemon': 180,
    'Maize': 120+30,
    'Mango (Raw-Ripe)': 140,
    'Mango': 150,
    'Mousambi(Sweet Lime)': 180,
    'Onion': 150,
    'Paddy(Dhan)(Common)': 180+30,
    'Ragi (Finger Millet)': 135+30,
    'Chili Red': 90+30,
    'Red Gram': 180+30,
    'Rubber': 365,
    'Sesamum': 140+30,
    'Sugarcane': 540,
    'Sunflower': 120+30,
    'Tapioca': 365,
    'Turmeric': 270
}

In [224]:
DISTRICT_COORDINATES = {
    "Ariyalur": {
        "lat": 11.1530559,
        "lon": 79.2585797
    },
    "Cuddalore": {
        "lat": 11.520207,
        "lon": 79.3396356
    },
    "Coimbatore": {
        "lat": 11.0018115,
        "lon": 76.9628425
    },
    "Salem": {
        "lat": 11.6469616,
        "lon": 78.2106958
    },
    "Vellore": {
        "lat": 12.9021849,
        "lon": 79.061104
    },
    "Erode": {
        "lat": 11.4905281,
        "lon": 77.3505274
    },
    "Theni": {
        "lat": 9.8692558,
        "lon": 77.4222974
    },
    "Namakkal": {
        "lat": 11.3033889,
        "lon": 78.118604
    },
    "Villupuram": {
        "lat": 11.9398285,
        "lon": 79.4945645
    },
    "Virudhunagar": {
        "lat": 9.4926289,
        "lon": 77.8631831
    },
    "Dindigul": {
        "lat": 10.4256896,
        "lon": 77.815664
    },
    "Thiruvannamalai": {
        "lat": 9.539867,
        "lon": 77.6197904
    },
    "Ramanathapuram": {
        "lat": 9.5205576,
        "lon": 78.5184635
    },
    "Tuticorin": {
        "lat": 8.8052602,
        "lon": 78.1452745
    },
    "Madurai": {
        "lat": 9.9261153,
        "lon": 78.1140983
    },
    "Karur": {
        "lat": 10.8217671,
        "lon": 78.3828654
    },
    "Dharmapuri": {
        "lat": 12.145613,
        "lon": 78.1132217
    },
    "Pudukkottai": {
        "lat": 10.2903072,
        "lon": 78.8173617
    },
    "Ranipet": {
        "lat": 12.9186243,
        "lon": 79.4081921
    },
    "Krishnagiri": {
        "lat": 12.5152075,
        "lon": 78.0093769
    },
    "Kancheepuram": {
        "lat": 12.8796051,
        "lon": 79.7042762
    },
    "Chengalpattu": {
        "lat": 12.6840886,
        "lon": 79.9836371
    },
    "The Nilgiris": {
        "lat": 11.4,
        "lon": 76.7
    },
    "Sivaganga": {
        "lat": 9.8486881,
        "lon": 78.4870461
    },
    "Thirupur": {
        "lat": 12.921745,
        "lon": 80.1375926
    },
    "Kallakuruchi": {
        "lat": 11.4248929,
        "lon": 78.2652132
    },
    "Thanjavur": {
        "lat": 10.659037,
        "lon": 79.2014278
    },
    "Nagapattinam": {
        "lat": 10.6026202,
        "lon": 79.7619348
    },
    "Thiruvarur": {
        "lat": 10.6878621,
        "lon": 79.4979437
    },
    "Thirupathur": {
        "lat": 10.59635,
        "lon": 79.56857
    },
    "Perambalur": {
        "lat": 11.2902641,
        "lon": 78.9300404
    },
    "Tenkasi": {
        "lat": 9.0933908,
        "lon": 77.4758373
    },
    "Thirunelveli": {
        "lat": 8.7150,
        "lon": 77.7656
    },
    "Thiruvellore" : {
        "lat": 13.1227,
        "lon": 79.9118
    },
    "Thiruchirappalli" : {
        "lat": 10.7905,
        "lon": 78.7047
    },
    "Nagercoil (Kannyiakumari)" : {
        "lat": 8.1833,
        "lon": 77.4119
    }
}

In [225]:
districts = {k: v for k, v in DISTRICT_COORDINATES.items() if k in df_agri['District Name'].unique()}
lookbacks = {k: v for k, v in CROP_LOOKBACK_DAYS.items() if k in df_agri['Commodity'].unique()}

print(f"Districts being processed for this file: {list(districts.keys())}")
print(f"Lookbacks being processed for this file: {lookbacks}")

Districts being processed for this file: ['Ariyalur', 'Cuddalore', 'Coimbatore', 'Salem', 'Vellore', 'Erode', 'Theni', 'Namakkal', 'Villupuram', 'Virudhunagar', 'Dindigul', 'Thiruvannamalai', 'Ramanathapuram', 'Tuticorin', 'Madurai', 'Karur', 'Dharmapuri', 'Pudukkottai', 'Ranipet', 'Krishnagiri', 'Kancheepuram', 'Chengalpattu', 'The Nilgiris', 'Sivaganga', 'Thirupur', 'Kallakuruchi', 'Thanjavur', 'Nagapattinam', 'Thiruvarur', 'Thirupathur', 'Perambalur', 'Tenkasi', 'Thirunelveli', 'Thiruvellore', 'Thiruchirappalli', 'Nagercoil (Kannyiakumari)']
Lookbacks being processed for this file: {'Mango (Raw-Ripe)': 140}


In [226]:
all_weather_data = []
unique_lookbacks = set(lookbacks.values())
start_date = df_agri['Price Date'].min() - pd.Timedelta(days=max(unique_lookbacks, default=0))
end_date = df_agri['Price Date'].max()

print(unique_lookbacks)

{140}


In [227]:
for district, coords in districts.items():
    print(f"Fetching weather data for {df_agri['Commodity'].unique()} in {district}...")
    
    weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={coords['lat']}&longitude={coords['lon']}&start_date={start_date.strftime('%Y-%m-%d')}&end_date={end_date.strftime('%Y-%m-%d')}&daily=temperature_2m_mean,precipitation_sum"
    response = requests.get(weather_url)
    
    if response.status_code == 200:
        df_weather = pd.DataFrame(response.json()['daily'])
        df_weather['Price Date'] = pd.to_datetime(df_weather['time'])
        df_weather['District Name'] = district

        for days in unique_lookbacks:
            df_weather[f'temp_mean_{days}d'] = df_weather['temperature_2m_mean'].rolling(window=days, min_periods=1).mean()
            df_weather[f'precip_sum_{days}d'] = df_weather['precipitation_sum'].rolling(window=days, min_periods=1).sum()
        
        all_weather_data.append(df_weather)
    print(response.status_code)
    time.sleep(10)
df_weather_master = pd.concat(all_weather_data)


Fetching weather data for ['Mango (Raw-Ripe)'] in Ariyalur...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Cuddalore...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Coimbatore...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Salem...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Vellore...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Erode...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Theni...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Namakkal...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Villupuram...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Virudhunagar...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Dindigul...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Thiruvannamalai...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Ramanathapuram...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Tuticorin...
200
Fetching weather data for ['Mango (Raw-Ripe)'] in Madurai...
20

In [228]:
print(df_weather_master)

           time  temperature_2m_mean  precipitation_sum Price Date  \
0    2024-01-30                 26.0                0.0 2024-01-30   
1    2024-01-31                 26.5                0.0 2024-01-31   
2    2024-02-01                 26.8                1.6 2024-02-01   
3    2024-02-02                 26.8                0.3 2024-02-02   
4    2024-02-03                 26.5                0.1 2024-02-03   
..          ...                  ...                ...        ...   
534  2025-07-17                 26.4                3.3 2025-07-17   
535  2025-07-18                 26.3                3.1 2025-07-18   
536  2025-07-19                 23.6               18.8 2025-07-19   
537  2025-07-20                 23.6               25.6 2025-07-20   
538  2025-07-21                 23.8               12.0 2025-07-21   

                 District Name  temp_mean_140d  precip_sum_140d  
0                     Ariyalur       26.000000              0.0  
1                     Ariya

In [229]:
df_merged = pd.merge(df_agri, df_weather_master, on=['District Name', 'Price Date'], how='left')
print(df_merged)

      District Name                    Market Name         Commodity  \
0        Dharmapuri    AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
1        Dharmapuri    AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
2        Dharmapuri    AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
3        Dharmapuri    AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
4        Dharmapuri    AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
...             ...                            ...               ...   
41308     Cuddalore  Viruthachalam(Farmers Market)  Mango (Raw-Ripe)   
41309     Cuddalore  Viruthachalam(Farmers Market)  Mango (Raw-Ripe)   
41310     Cuddalore  Viruthachalam(Farmers Market)  Mango (Raw-Ripe)   
41311     Cuddalore  Viruthachalam(Farmers Market)  Mango (Raw-Ripe)   
41312     Cuddalore  Viruthachalam(Farmers Market)  Mango (Raw-Ripe)   

                Variety  Grade  Min Price (Rs./Quintal)  \
0      Mango - Raw-Ripe  Local                   1200.0   
1      Mango - Ra

In [230]:
def get_correct_lookback_value(row, feature_prefix, lookback_dict):
    commodity = row['Commodity']
    # ** NEW: Check if the commodity exists in our lookback dictionary **
    if commodity in lookback_dict:
        lookback_days = lookback_dict[commodity]
        column_name = f"{feature_prefix}_{lookback_days}d"
        return row[column_name]
    return None

df_merged['lookback_temp_mean'] = df_merged.apply(get_correct_lookback_value, feature_prefix='temp_mean', lookback_dict=lookbacks, axis=1)
df_merged['lookback_precip_sum'] = df_merged.apply(get_correct_lookback_value, feature_prefix='precip_sum', lookback_dict=lookbacks, axis=1)


In [231]:
print(df_merged.head())

  District Name                  Market Name         Commodity  \
0    Dharmapuri  AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
1    Dharmapuri  AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
2    Dharmapuri  AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
3    Dharmapuri  AJattihalli(Farmers Market)  Mango (Raw-Ripe)   
4    Dharmapuri  AJattihalli(Farmers Market)  Mango (Raw-Ripe)   

            Variety  Grade  Min Price (Rs./Quintal)  Max Price (Rs./Quintal)  \
0  Mango - Raw-Ripe  Local                   1200.0                   1500.0   
1  Mango - Raw-Ripe  Local                   1400.0                   1600.0   
2  Mango - Raw-Ripe  Local                   1400.0                   1600.0   
3  Mango - Raw-Ripe  Local                   1400.0                   1600.0   
4  Mango - Raw-Ripe  Local                   1400.0                   1600.0   

   Modal Price (Rs./Quintal) Price Date  Day Of Week        time  \
0                     1500.0 2025-07-13            6  

In [232]:
final_columns = [
    'District Name', 'Market Name', 'Commodity', 'Variety', 'Grade', 'Min Price (Rs./Quintal)', 
    'Max Price (Rs./Quintal)', 'Modal Price (Rs./Quintal)', 'Price Date', 'Day Of Week',
    'lookback_temp_mean', 'lookback_precip_sum'
]

df_final = df_merged[[col for col in final_columns if col in df_merged.columns]].copy()


In [ ]:
df_final.to_csv('../Weather_Merged_CSVs/Maize-Raw-Ripe.csv', index=False)